In [1]:
import pandas as pd
# xgboost
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
# scores
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, make_scorer
# kfold
from sklearn.model_selection import KFold
# gridsearch
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
# matplotlib
import matplotlib.pyplot as plt
# datetime
import datetime as dt
# numpy
import numpy as np
import sys
sys.path.insert(1, 'C:/Users/berna/Desktop/compes/DATATHON UPC FME 2023/healthcare_challenge/')
from utils import *

In [2]:
# load train data
df_train = pd.read_csv('../data/consumo_no23.csv')
df_test = pd.read_csv('../data/consumo_23.csv')
df_train = preprocess_cosumo(df_train)
df_test = preprocess_cosumo(df_test)

asjfkjdsfljds
asjfkjdsfljds


In [3]:
# train_test_split
X = df_train.drop(['STACKS_COMPRATS'], axis = 1)
y = df_train['STACKS_COMPRATS']
folds = KFold(n_splits=10, shuffle=True, random_state=42)

In [4]:
# timer
def timer(start_time=None):
    if not start_time:
        start_time = dt.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((dt.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))



In [5]:
# gridsearch for xgboost
param_grid = {
    'n_estimators': [100, 150, 200],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.2, 0.5, 0.8],
    # 'colsample_bytree': [0.8, 0.9, 1.0],
    # 'gamma': [0, 0.1, 0.2],
    # 'min_child_weight': [1, 3, 5],
    'reg_alpha': [0.1, 0.2, 0.5],
    'reg_lambda': [0.1, 0.2, 0.5],
    # 'objective': ['reg:squarederror','reg:squaredlogerror','reg:linear']
}

In [6]:
# xgboost
model = xgb.XGBRegressor(nthread=10, objective='reg:squarederror')
# grid_search = RandomizedSearchCV(estimator=xgb, param_distributions=param_grid, cv=folds, n_jobs=10, verbose=2, scoring=('neg_mean_squared_error', 'neg_mean_absolute_error', 'r2'))
grid_search = RandomizedSearchCV(estimator=model, param_distributions=param_grid, cv=folds, n_jobs=10, verbose=1, scoring='neg_mean_squared_error')
# start_time = timer(None)
grid_search.fit(X, y)
# timer(start_time) # timing ends here for "start_time" variable

Fitting 10 folds for each of 10 candidates, totalling 100 fits


RandomizedSearchCV(cv=KFold(n_splits=10, random_state=42, shuffle=True),
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, gpu_id=None,
                                          grow_policy=None,
                                          importance_type=No...
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          n_estimators=100, n_jobs=None,
                                          nthread=10, num_parallel_tree=None,
                                          predictor=None, ...),
                   n_jobs=10,
                   param_distributions={'learning_rate': [0.01, 0.05, 0.1, 0.2],
                                        'max_depth': [3, 5, 7],
                                        'n_estimators': [100, 150, 200],
                                        'reg_alpha': [0.1, 0.2, 0.5],
                                        'reg_lambda': [0.1, 0.2, 0.5],
                                        'subsample': [0.2, 0.5, 0.8]},
                   scoring='neg_mean_squared_error', verbose=1)

In [7]:
grid_search.best_params_

{'subsample': 0.2,
 'reg_lambda': 0.2,
 'reg_alpha': 0.1,
 'n_estimators': 200,
 'max_depth': 7,
 'learning_rate': 0.2}

In [8]:
# metrics
X_test = df_test.drop(['STACKS_COMPRATS'], axis = 1)
y_test = df_test['STACKS_COMPRATS']
preds = grid_search.predict(X)

mse = mean_squared_error(y, preds)
print('mse: ', mse)
mae = mean_absolute_error(y, preds)
print('mae: ', mae)
r2 = r2_score(y, preds)
print('r2: ', r2)


mse:  0.11801664032446788
mae:  0.18884997293386546
r2:  0.9999086264321915


In [23]:
cv_results = grid_search.cv_results_
cv_results.keys()

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_subsample', 'param_reg_lambda', 'param_reg_alpha', 'param_n_estimators', 'param_max_depth', 'param_learning_rate', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'split5_test_score', 'split6_test_score', 'split7_test_score', 'split8_test_score', 'split9_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])

In [24]:
cv_results['split0_test_score']

array([ -71.06260643,   -1.48169053,   -1.04355218,   -2.49338462,
         -2.35966534, -229.44346397,   -3.98889338,   -0.94846635,
         -3.15867935,   -3.74957348])

In [22]:
# Access the results
cv_results = grid_search.cv_results_
for param, score, fold in zip(cv_results["params"], cv_results["mean_test_score"], cv_results["rank_test_score"]):
# Print the results for each fold
for i in range(grid_search.n_splits_):
    print(f"Results for fold {i + 1}:")
    for key in cv_results.keys():
        if key.startswith(f"split{i}_"):
            print(f"{key}: {cv_results[key][grid_search.best_index_]}")
    print("\n")

[{'subsample': 0.8, 'reg_lambda': 0.2, 'reg_alpha': 0.2, 'n_estimators': 200, 'max_depth': 3, 'learning_rate': 0.01}, {'subsample': 0.2, 'reg_lambda': 0.2, 'reg_alpha': 0.1, 'n_estimators': 200, 'max_depth': 7, 'learning_rate': 0.2}, {'subsample': 0.5, 'reg_lambda': 0.2, 'reg_alpha': 0.1, 'n_estimators': 150, 'max_depth': 7, 'learning_rate': 0.1}, {'subsample': 0.5, 'reg_lambda': 0.2, 'reg_alpha': 0.1, 'n_estimators': 150, 'max_depth': 5, 'learning_rate': 0.05}, {'subsample': 0.2, 'reg_lambda': 0.1, 'reg_alpha': 0.2, 'n_estimators': 200, 'max_depth': 5, 'learning_rate': 0.05}, {'subsample': 0.5, 'reg_lambda': 0.5, 'reg_alpha': 0.2, 'n_estimators': 100, 'max_depth': 5, 'learning_rate': 0.01}, {'subsample': 0.2, 'reg_lambda': 0.5, 'reg_alpha': 0.2, 'n_estimators': 150, 'max_depth': 3, 'learning_rate': 0.1}, {'subsample': 0.8, 'reg_lambda': 0.2, 'reg_alpha': 0.1, 'n_estimators': 100, 'max_depth': 7, 'learning_rate': 0.1}, {'subsample': 0.8, 'reg_lambda': 0.5, 'reg_alpha': 0.2, 'n_estimato

In [10]:
start_time = timer(None)
[8 for i in range(100000)]
timer(start_time) # timing ends here for "start_time" variable


AttributeError: module 'datetime' has no attribute 'now'